In [1]:
from tqdm import tqdm
import pandas as pd
import re

In [2]:
df = pd.read_csv('posts.csv')

In [3]:
# Check if id is unique
print(df['id'].count(), df['id'].nunique())

727182 727182


In [4]:
df = df.set_index('id')
df.head()

,created_utc,subreddit,author,title,selftext,permalink,url
id,,,,,,,
7nca41,1514768665,wallstreetbets,alreadyinuse5000,Blowing versus sucking,AAPL just entered a contract to purchase 51 of...,/r/wallstreetbets/comments/7nca41/blowing_vers...,https://www.reddit.com/r/wallstreetbets/commen...
7ncxcn,1514776674,pcmasterrace,castatech,Why is it that the new Windows 10 picture view...,What kind of an upgrade was this? It just suck...,/r/pcmasterrace/comments/7ncxcn/why_is_it_that...,https://www.reddit.com/r/pcmasterrace/comments...
7ndtyy,1514789593,TIMESINDIAauto,AutoNewsAdmin,[Business] - New year looks grim for McD’s staff,NaN,/r/TIMESINDIAauto/comments/7ndtyy/business_new...,https://timesofindia.indiatimes.com/business/i...
7ndu9l,1514789727,AutoNewspaper,AutoNewspaperAdmin,[Business] - New year looks grim for McD’s sta...,NaN,/r/AutoNewspaper/comments/7ndu9l/business_new_...,https://timesofindia.indiatimes.com/business/i...
7neolz,1514805270,teslamotors,AutoModerator,Bi-weekly TSLA Investor Thread,This will post every other Monday (EST) at 6AM...,/r/teslamotors/comments/7neolz/biweekly_tsla_i...,https://www.reddit.com/r/teslamotors/comments/...


In [6]:
keywords = ['tsla', 'aapl', 'msft', 'gme', 'nvda', 'nflx', 'mcd']
pattern = r'(?i)\b(\$?' + '|'.join(keywords) + r')\b'

df['title'] = df['title'].astype(str)
df['selftext'] = df['selftext'].astype(str)

dfs = []
for row in tqdm(df.itertuples(), total=len(df)):
    title, body, created_utc = row.title, row.selftext, row.created_utc
    symbols_in_post = set()

    for match in re.finditer(pattern, title + " " + body):
        symbol = match.group().lstrip("$").lower()
        symbols_in_post.add(symbol)

    for symbol in symbols_in_post:
        index_row = pd.DataFrame({
            'id': [row.Index],
            'stock_symbol': [symbol],
            'created_utc': [created_utc]
        })
        dfs.append(index_row)

stock_index = pd.concat(dfs, ignore_index=True)

100%|██████████| 727182/727182 [03:29<00:00, 3463.34it/s]


In [7]:
stock_index.head()

,id,stock_symbol,created_utc
0,7nca41,aapl,1514768665
1,7ncxcn,msft,1514776674
2,7ndtyy,mcd,1514789593
3,7ndu9l,mcd,1514789727
4,7neolz,tsla,1514805270


In [8]:
stock_index.to_csv('stock_index.csv', index=False)